In [4]:
library(tidyverse)
library(repr)
library(tidymodels)
options(repr.matrix.max.rows = 6)
source('tests.R')
source('cleanup.R')

Warning message in file(filename, "r", encoding = encoding):
“cannot open file 'tests.R': No such file or directory”


ERROR: Error in file(filename, "r", encoding = encoding): cannot open the connection


In [ ]:
train <- read_csv("train.csv") %>% mutate(Survived = as_factor(Survived), DirectRelatives = SibSp + Parch) %>% mutate(across(where(is.numeric),scale)) %>%  drop_na(-Cabin)
summary(train)

In [5]:
titanic_recipe <- recipe(Survived ~ Age + Fare , data = train) %>%
   step_scale(all_predictors()) %>%
    step_center(all_predictors())
titanic_tune <- nearest_neighbor(weight_func = "rectangular", neighbors = tune()) %>%
      set_engine("kknn") %>%
      set_mode("classification")
titanic_vfold <- vfold_cv(train, v = 5, strata = Survived)
titanic_results <- workflow() %>%
      add_recipe(titanic_recipe) %>%
      add_model(titanic_tune) %>%
      tune_grid(resamples = titanic_vfold, grid = 50) %>%
      collect_metrics()
accuracies <- titanic_results%>%
filter(.metric == 'accuracy')

ERROR: Error in is_tibble(data): object 'train' not found


In [ ]:
titanic_plot <- ggplot(accuracies, aes(x = neighbors, y = mean)) +
geom_point() +
geom_line() +
labs(x = 'Neighbors', y = 'Accuracy Estimate')
titanic_plot

In [2]:
options(repr.plot.width = 10, repr.plot.height = 8) 
titanic_plot <- train %>% 
        ggplot(aes(x = Age, y = Fare)) +
        geom_point(aes(colour = Survived, shape = Sex)) + 
        labs(x = "Age", y = "Fare Value", 
             colour = "Survived", shape = "Sex") +
        ggtitle("Relationship Between Survival and Fare value, Age, and Sex")
titanic_plot

ERROR: Error in eval(lhs, parent, parent): object 'train' not found


Extra Code:

In [ ]:
set.seed(1999)

titanic_recipe <- recipe(Survived ~ Age + Fare, data = train) %>%
    step_scale(all_predictors()) %>%
    step_center(all_predictors())
titanic_spec <- nearest_neighbor(weight_func = "rectangular", neighbors = 3) %>%
       set_engine("kknn") %>%
       set_mode("classification")
titanic_fit <- workflow() %>%
       add_recipe(titanic_recipe) %>%
       add_model(titanic_spec) %>%
       fit(data = train)
titanic_test_predictions <- predict(titanic_fit, train) %>%
    bind_cols(train)
titanic_prediction_accuracy <- titanic_test_predictions %>%
         metrics(truth = Survived, estimate = .pred_class)
titanic_prediction_accuracy

In [ ]:
titanic_vfold <- vfold_cv(train, v = 5, strata = Survived)
titanic_resample_fit <- workflow() %>%
       add_recipe(titanic_recipe) %>%
       add_model(titanic_spec) %>%
       fit_resamples(resamples = titanic_vfold)
titanic_metrics <- workflow() %>%
       add_recipe(titanic_recipe) %>%
       add_model(titanic_spec) %>%
       fit_resamples(resamples = titanic_vfold) %>%
       collect_metrics()
titanic_metrics

titanic_tune <- nearest_neighbor(weight_func = "rectangular", 
                             neighbors = tune()) %>%
  set_engine("kknn") %>%
  set_mode("classification")
knn_results <- workflow() %>%
       add_recipe(titanic_recipe) %>%
       add_model(titanic_tune) %>%
       tune_grid(resamples = titanic_vfold, grid = 10) %>%
       collect_metrics()
knn_results

accuracies <- knn_results %>% 
       filter(.metric == "accuracy")
accuracy_versus_k <- ggplot(accuracies, aes(x = neighbors, y = mean))+
       geom_point() +
       geom_line() +
       labs(x = "Neighbors", y = "Accuracy Estimate") +
       scale_x_continuous(breaks = seq(0, 14, by = 1)) +  # adjusting the x-axis
       scale_y_continuous(limits = c(0.4, 1.0)) # adjusting the y-axis
accuracy_versus_k